## reference
https://www.kaggle.com/robikscube/helper-code-helmet-mapping-deepsort

## import

### private library

In [1]:
from external_lib.NFLlib.score import NFLAssignmentScorer, check_submission
from external_lib.NFLlib.features import add_track_features

from lib.math.coodinate import generate_rotation_maxrix
from lib.math.coodinate import convert_3d_to_2d

from lib.noglobal import noglobal

### public library

In [2]:
import numpy as np
import pandas as pd
import itertools
import glob
import os
import cv2
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm
from multiprocessing import Pool
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import random

## paramters

In [3]:
n_debug_samples = 120
random_state = 42
CONF_THRE = 0.3
max_iter = 1000
DIG_STEP = 3
DIG_MAX = 42
base_dir = "/work/data/input/nfl-health-and-safety-helmet-assignment"

debug = True


paramater = {};
paramater["n_debug_samples"] = n_debug_samples
paramater["random_state"] = random_state
paramater["CONF_THRE"] = CONF_THRE
paramater["max_iter"] = max_iter
paramater["DIG_STEP"] = DIG_STEP
paramater["DIG_MAX"] = random_state


## file reading

In [4]:
labels = pd.read_csv(f"{base_dir}/train_labels.csv")

if debug:
    tracking = pd.read_csv(f'{base_dir}/train_player_tracking.csv')
    helmets = pd.read_csv(f'{base_dir}/train_baseline_helmets.csv')
else:
    tracking = pd.read_csv(f'{base_dir}/test_player_tracking.csv')
    helmets = pd.read_csv(f'{base_dir}/test_baseline_helmets.csv')

## preprocessing

### function definition

In [5]:
def add_cols(df):
    df['game_play'] = df['video_frame'].str.split('_').str[:2].str.join('_')
    if 'video' not in df.columns:
        df['video'] = df['video_frame'].str.split('_').str[:3].str.join('_') + '.mp4'
    return df

### both train and test

### function difinition

In [6]:
tracking = add_track_features(tracking)
tracking["z"] = 6.0

if debug:
    helmets = add_cols(helmets)
    labels = add_cols(labels)    
    
    # Select `n_debug_samples` worth of videos to debug with
    sample_videos = labels['video'].drop_duplicates() \
        .sample(n_debug_samples, random_state=random_state).tolist()
    sample_gameplays = ['_'.join(x.split('_')[:2]) for x in sample_videos]
    tracking = tracking[tracking['game_play'].isin(sample_gameplays)]
    helmets = helmets[helmets['video'].isin(sample_videos)]
    labels = labels[labels['video'].isin(sample_videos)]

    
helmets["frame"] = helmets["video_frame"].apply(lambda x:x.split("_")[3])
helmets["video_frame_without_position"] = helmets["video_frame"].apply(lambda x: "_".join( x.split("_")[:2] + [x.split("_")[3]]   )  )
    


for key,hel_each_df in helmets.groupby("game_play"):    
    min_frame = hel_each_df["frame"].astype(int).min()
    max_frame = hel_each_df["frame"].astype(int).max()
    tracking = tracking[tracking["est_frame"] > (min_frame-5)]
    tracking = tracking[tracking["est_frame"] < max_frame+5]  


## main

### first part (basic allocation)

#### function definition

In [7]:
submission_df_list = []
df_list = list(helmets.groupby('video_frame'))
df_list = [list(element) + [tracking,paramater] for element in df_list];

### 実行

### 

## Deep Sort

### read submission file

In [8]:
from external_lib.deep_sort_pytorch.utils.parser import get_config
from external_lib.deep_sort_pytorch.deep_sort import DeepSort

In [9]:
def compute_color_for_id(label):
    """
    Simple function that adds fixed color depending on the id
    """
    palette = (2 ** 11 - 1, 2 ** 15 - 1, 2 ** 20 - 1)

    color = [int((p * (label ** 2 - label + 1)) % 255) for p in palette]
    return tuple(color)

def plot_one_box(x, im, color=None, label=None, line_thickness=3):
    # Plots one bounding box on image 'im' using OpenCV
    assert im.data.contiguous, 'Image not contiguous. Apply np.ascontiguousarray(im) to plot_on_box() input image.'
    tl = line_thickness or round(0.002 * (im.shape[0] + im.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(im, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label: 
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(im, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(im, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)
    return im

In [10]:

def deepsort_helmets(video_data,
                     video_dir,
                     deepsort_config='/work/config/deepsort.yaml',
                     plot=False,
                     plot_frames=[]):
    
    # Setup Deepsort
    cfg = get_config()
    cfg.merge_from_file(deepsort_config)    
    deepsort = DeepSort(cfg.DEEPSORT.REID_CKPT,
                        max_dist=cfg.DEEPSORT.MAX_DIST,
                        min_confidence=cfg.DEEPSORT.MIN_CONFIDENCE,
                        nms_max_overlap=cfg.DEEPSORT.NMS_MAX_OVERLAP,
                        max_iou_distance=cfg.DEEPSORT.MAX_IOU_DISTANCE,
                        max_age=cfg.DEEPSORT.MAX_AGE,
                        n_init=cfg.DEEPSORT.N_INIT,
                        nn_budget=cfg.DEEPSORT.NN_BUDGET,
                        use_cuda=True)
    
    # Run through frames.
    video_data = video_data.sort_values('frame').reset_index(drop=True)
    ds = []
    for frame, d in tqdm(video_data.groupby(['frame']), total=video_data['frame'].nunique()):
        d['x'] = (d['left'] + round(d['width'] / 2))
        d['y'] = (d['top'] + round(d['height'] / 2))

        xywhs = d[['x','y','width','height']].values

        cap = cv2.VideoCapture(f'{video_dir}/{myvideo}.mp4')
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame-1) # optional
        
        ##シーケンス
        success, image = cap.read()
        
        # 画像の色の順番を変更（BGR to RGB）
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        confs = np.ones([len(d),])
        clss =  np.zeros([len(d),])
        
        outputs = deepsort.update(xywhs, confs, clss, image)

        preds_df = pd.DataFrame(outputs, columns=['left','top','right','bottom','deepsort_cluster','class'])
        if len(preds_df) > 0:
            # TODO Fix this messy merge
            d = pd.merge_asof(d.sort_values(['left','top']),
                              preds_df[['left','top','deepsort_cluster']] \
                              .sort_values(['left','top']), on='left', suffixes=('','_deepsort'),
                              direction='nearest')
        ds.append(d)
    dout = pd.concat(ds)
    return dout


def add_deepsort_label_col(out):
    # Find the top occuring label for each deepsort_cluster
    sortlabel_map = out.groupby('deepsort_cluster')['label'].value_counts() \
        .sort_values(ascending=False).to_frame() \
        .rename(columns={'label':'label_count'}) \
        .reset_index() \
        .groupby(['deepsort_cluster']) \
        .first()['label'].to_dict()
    # Find the # of times that label appears for the deepsort_cluster.
    sortlabelcount_map = out.groupby('deepsort_cluster')['label'].value_counts() \
        .sort_values(ascending=False).to_frame() \
        .rename(columns={'label':'label_count'}) \
        .reset_index() \
        .groupby(['deepsort_cluster']) \
        .first()['label_count'].to_dict()
    
    out['label_deepsort'] = out['deepsort_cluster'].map(sortlabel_map)
    out['label_count_deepsort'] = out['deepsort_cluster'].map(sortlabelcount_map)

    return out


def score_vs_deepsort(myvideo, out, labels):
    # Score the base predictions compared to the deepsort postprocessed predictions.
    myvideo_mp4 = myvideo + '.mp4'
    labels_video = labels.query('video == @myvideo_mp4')
    scorer = NFLAssignmentScorer(labels_video)
    out_deduped = out.groupby(['video_frame','label']).first().reset_index()
    base_video_score = scorer.score(out_deduped)
    
    out_preds = out.drop('label', axis=1).rename(columns={'label_deepsort':'label'})
    print(out_preds.shape)
    out_preds = out_preds.groupby(['video_frame','label']).first().reset_index()
    print(out_preds.shape)
    deepsort_video_score = scorer.score(out_preds)
    print(f'{base_video_score:0.5f} before --> {deepsort_video_score:0.5f} deepsort')

## main

In [11]:
submission_df_v2 = pd.read_csv("submission-baseline_upgrade.csv")

In [12]:
# Add video and frame columns to submission.
submission_df = submission_df_v2;

submission_df['video'] = submission_df['video_frame'].str.split('_').str[:3].str.join('_')
submission_df['frame'] = submission_df['video_frame'].str.split('_').str[-1].astype('int')

labels = pd.read_csv(f"{base_dir}/train_labels.csv")

if debug:
    video_dir = f"{base_dir}/train/"
else:
    video_dir = f"{base_dir}/test/"

    
    

# Loop through test videos and apply. If in debug mode show the score change.
outs = []
counter = 0;
for myvideo, video_data in tqdm(submission_df.groupby('video'), total=submission_df['video'].nunique()):
    counter = counter + 1;
    
    if myvideo == "57788_000781_Endzone":
        continue;
        
    
    if debug:
        # Plot deepsort labels when in debug mode.
        out = deepsort_helmets(video_data, video_dir, plot_frames=[10, 150, 250])
    else:
        out = deepsort_helmets(video_data, video_dir)        
    out = add_deepsort_label_col(out)
    outs.append(out)
    if debug:        
        pass;
        #score_vs_deepsort(myvideo, out, labels)
        
    
submission_deepsort = pd.concat(outs).copy()

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/472 [00:00<?, ?it/s]

  0%|          | 0/471 [00:00<?, ?it/s]

  0%|          | 0/615 [00:00<?, ?it/s]

  0%|          | 0/616 [00:00<?, ?it/s]

...

  0%|          | 0/350 [00:00<?, ?it/s]

  0%|          | 0/549 [00:00<?, ?it/s]

  0%|          | 0/548 [00:00<?, ?it/s]

  0%|          | 0/473 [00:00<?, ?it/s]

  0%|          | 0/473 [00:00<?, ?it/s]

In [13]:
print(myvideo)

58107_004362_Sideline


In [16]:
sub = submission_deepsort.copy()
sub = sub.drop('label', axis=1).rename(columns={'label_deepsort':'label'}).groupby(['video_frame','label']).first().reset_index()

In [20]:
from lib.noglobal import noglobal

@noglobal()
def merge_base_and_deepsort(base_sub,deepsort_sub, erase_duplicated = False,random_allocation=False):
    if (erase_duplicated and random_allocation):
        raise Exception("asdf")
    
    
    unique_player_list = base_sub["label"].unique().tolist()
    
    ret_df = base_sub.copy()
    columns = ret_df.columns
        
    ## key列の追加
    deepsort_sub["key"] = deepsort_sub["video_frame"] + deepsort_sub["left"].astype(str) + deepsort_sub["width"].astype(str)+ deepsort_sub["top"].astype(str) + deepsort_sub["height"].astype(str)
    ret_df["key"] = ret_df["video_frame"] + ret_df["left"].astype(str) + ret_df["width"].astype(str)+ ret_df["top"].astype(str) + ret_df["height"].astype(str)
    
    ## deepsortの推定結果をhashmapに加える。
    deepsort_expectation_hashmap = {};
    for key,label in zip(deepsort_sub["key"],deepsort_sub["label"]):
        deepsort_expectation_hashmap[key] = label
        
    ret_df["label"] = ret_df.apply(lambda x:  deepsort_expectation_hashmap[x["key"]] if (x["key"] in deepsort_expectation_hashmap.keys()) else x["label"] ,axis=1)
    ## 1: Deepsort result, 0: base_sub result
    ret_df["isDeepSpecuration"] = ret_df.apply(lambda x:  1 if (x["key"] in deepsort_expectation_hashmap.keys()) else 0,axis=1)
    
    
    if (erase_duplicated):
        bec = [];
        for key,each_df in ret_df.groupby("video_frame"):
            each_df = each_df.sort_values("isDeepSpecuration")
                        
            unique_df = each_df[~each_df.duplicated(subset="label",keep='first')]
            
            bec.append(unique_df)
    
        ret_df = pd.concat(bec)
        
    
    
    if (random_allocation):
        bec = [];
        for key,each_df in ret_df.groupby("video_frame"):
            each_df = each_df.sort_values("isDeepSpecuration")
                        
            unique_df = each_df[~each_df.duplicated(subset="label",keep='first')]
            duplicated_df = each_df[each_df.duplicated(subset="label",keep='first')]
            
            notcandidate = unique_df["label"].unique().tolist()
            candidate_df = [ ele   for ele in unique_player_list if (ele not in notcandidate) ]                        
            
            sample = random.sample(candidate_df, duplicated_df.shape[0])
            
            if ( len(sample) != 0):
                duplicated_df["label"] = sample
                unique_df = unique_df.append(duplicated_df)
            
            bec.append(unique_df)
    
        ret_df = pd.concat(bec)
    
    
    deepsort_sub = deepsort_sub.drop("key",axis=1)
    ret_df = ret_df.drop("key",axis=1)
    
    return ret_df;
            

In [21]:
Union_sub = merge_base_and_deepsort(submission_df_v2,sub,erase_duplicated=False,random_allocation=True);
#Union_sub = merge_base_and_deepsort(ext_sub,ext_deepsort,erase_duplicated=False,random_allocation=True);
Union_sub = Union_sub.reset_index(drop=True)

<string>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [23]:
if debug:
    scorer = NFLAssignmentScorer(labels)
    baseline_score = scorer.score(Union_sub)
    print(f"validation score {baseline_score:0.4f}")

validation score 0.3963
